In [1]:
! pip install cma

     |████████████████████████████████| 230 kB 699 kB/s eta 0:00:01


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import timeit

import numpy as np
import cma
from IPython.display import display, clear_output

from anguilla.optimizers.cma import CMA, StrategyParameters, StoppingConditions
from anguilla.fitness.benchmark import Sphere, SumSquares

In [3]:
def test_with_pycma(n=4):
    fitness_function = Sphere()
    initial_point = fitness_function.propose_initial_point(n)
    initial_sigma = 0.3
    cma.fmin(fitness_function, initial_point, initial_sigma, options={'CMA_Active': 0})

test_with_pycma()

(4_w,8)-CMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=475004, Thu Nov 26 04:10:53 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      8 4.092089506002811e+00 1.0e+00 3.55e-01  3e-01  4e-01 0:00.0
    2     16 2.474184999512569e+00 1.1e+00 4.74e-01  5e-01  5e-01 0:00.0
    3     24 1.451724118995040e+00 1.2e+00 6.46e-01  6e-01  7e-01 0:00.0
  100    800 1.035602852733542e-13 1.5e+00 1.30e-05  3e-07  4e-07 0:00.4
  113    904 4.546229930830256e-15 1.6e+00 4.27e-06  6e-08  9e-08 0:00.5
termination on tolfun=1e-11 (Thu Nov 26 04:10:58 2020)
final/bestever f-value = 2.895012e-15 2.895012e-15
incumbent solution: [1.9294181211161087e-08, -7.933897761477204e-09, -4.738194833547888e-08, 1.4654381448775531e-08]
std deviation: [8.141071626222936e-08, 7.048185732234207e-08, 6.347768198276708e-08, 8.797508817776637e-08]


In [4]:
def test_with_prototype(n=4):
    parameters = StrategyParameters(n=n)
    conditions = StoppingConditions(n=n,
                                    population_size=parameters.population_size,
                                    ftarget=1e-14,
                                    )
    fitness_function = Sphere()
    initial_point = fitness_function.propose_initial_point(n)
    optimizer = CMA(initial_point=initial_point,
                    initial_sigma=0.3,
                    strategy_parameters=parameters,
                    stopping_conditions=conditions)
    while not optimizer.stop():
        candidate_solutions = optimizer.ask()
        function_values = fitness_function(candidate_solutions.T)
        ranked_indices = optimizer.rank(candidate_solutions, function_values)
        optimizer.tell(candidate_solutions, ranked_indices)
        if (optimizer._fevals % 20) == 0:
            display(optimizer._fevals, optimizer._best_value, optimizer._best_solution)

In [5]:
test_with_prototype(n=4)

20

4.189925594216071

array([-1.88365048,  0.00288588, -0.73233761,  0.3247457 ])

40

0.480606102934387

...

2.750956369628505e-14

array([ 1.09486538e-07, -1.11237871e-07, -2.55003421e-08, -4.99812997e-08])

800

6.418551269408425e-15

array([-2.18972851e-08,  8.55731258e-09,  2.73564635e-09,  7.65398511e-08])